In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
basket = pd.read_csv('basket_dataset/Market_Basket_Optimisation.csv', header=None)

In [3]:
basket.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
table = []

In [5]:
list_products = np.array(list(set(basket.values.flatten())))

In [6]:
list_products = list_products[list_products != 'nan']

In [7]:
try:
    for b in basket.loc:
        table.append(np.isin(list_products, b.values).astype(int))
except:
    pass

In [8]:
basket_encoded = pd.DataFrame(table, columns=list_products)

In [9]:
basket_encoded.head()

,green grapes,antioxydant juice,pickles,hand protein bar,cottage cheese,light cream,pet food,toothpaste,cooking oil,shampoo,...,shallot,ground beef,parmesan cheese,mineral water,milk,avocado,chicken,fresh bread,cake,red wine
0,1,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [10]:
basket_encoded = basket_encoded[basket_encoded.columns[basket_encoded.mean() > 0.05]][basket_encoded.sum(axis=1) > 1]

In [11]:
list_products = basket_encoded.columns

In [12]:
def generate_permutations(columns, n):
    if n == 1:
        return [(k,) for k in columns]
    permutations = []
    for i, c in enumerate(columns):
        permutations.extend([(c,) + p for p in generate_permutations(columns[i+1:], n-1)])
    return permutations

def generate_all_possible_combinations(columns, n=3):
    combinations = []
    for i in range(1, n+1):
        combinations.extend(generate_permutations(columns, i))
    return combinations 

In [13]:
generate_all_possible_combinations([1,2,3,4], 3)

[(1,),
 (2,),
 (3,),
 (4,),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4)]

In [14]:
all_possible_basket_combinations = generate_all_possible_combinations(list_products, n=3)

In [15]:
def calculate_combination_probability(df, combination):
    logic = None
    for product in combination:
        if logic is None:
            logic = df[product] == 1
            continue
        logic &= (df[product] == 1)
    return len(df[logic]) / len(df)

In [16]:
probabilities = {}
for combination in all_possible_basket_combinations:
    probabilities[frozenset(combination)] = calculate_combination_probability(basket_encoded, combination)

In [17]:
def generate_rules(combination):
    possible_comb = generate_all_possible_combinations(combination)
    rules = []
    for if_ in possible_comb:
        for then_ in possible_comb:
            if set(if_).intersection(set(then_)).__len__() != 0:
                continue
            rules.append((if_, then_))
    return rules

In [18]:
rules_generator = list(filter(lambda x: len(x) > 1, all_possible_basket_combinations))

In [19]:
rules = []

for combination in rules_generator:
    rules.extend(generate_rules(combination))

In [20]:
rules = list(set(rules))

In [21]:
rules_confidence = {}
lift = {}
support = {}

In [22]:
for (if_, then_) in rules:
    support[(if_, then_)] = probabilities[frozenset(if_ + then_)]
    if probabilities[frozenset(if_)] != 0:
        rules_confidence[(if_, then_)] = probabilities[frozenset(if_ + then_)] / probabilities[frozenset(if_)]
        if probabilities[frozenset(then_)] != 0:
            lift[(if_, then_)] = probabilities[frozenset(if_ + then_)] / (probabilities[frozenset(if_)] * probabilities[frozenset(then_)])

In [23]:
def association_rules(support_lower=0.05, confidence_lower=0.9, lift_lower=1):
    rules_list = list(lift.keys())
    return list(filter(lambda x: support[x] > support_lower and rules_confidence[x] > confidence_lower and lift[x] > lift_lower, rules_list))

In [24]:
pd.DataFrame(association_rules(support_lower=0.0005))

""


### Using Decision Tree to help with rules selection

For example, we want to see milk pattern

In [25]:
from sklearn.tree import DecisionTreeClassifier

In [26]:
tree = DecisionTreeClassifier(max_depth=5)

In [27]:
interested_product = 'milk'

In [28]:
y = basket_encoded[interested_product]
X = basket_encoded.drop(interested_product, axis=1)

In [29]:
tree.fit(X, y)

DecisionTreeClassifier(max_depth=5)

In [30]:
def get_association_rules_from_tree(tree, feature_names, out):
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]

    # get ids of child nodes
    idx = np.argwhere(left == -1)[:,0]     

    def recurse(left, right, child, lineage=None):
        if lineage is None:
            buy = np.argmax(tree.tree_.value[child]) 
            lineage = [buy]
        if child in left:
            parent = np.where(left == child)[0].item()
            split = 'l'
        else:
            parent = np.where(right == child)[0].item()
            split = 'r'

        lineage.append((parent, split, threshold[parent], features[parent]))

        if parent == 0:
            lineage.reverse()
            return lineage
        else:
            return recurse(left, right, parent, lineage)
    
    rules = []

    currule = tuple()
    for child in idx:
        for node in recurse(left, right, child):
            if isinstance(node, np.int64):
                if node == 1:
                    rules.append((currule, out))
                currule = tuple()
                continue
            if node[1] == 'r':
                currule = (node[3],) + currule
    return rules


In [31]:
rules = get_association_rules_from_tree(tree, tree.feature_names_in_, out=('milk',))

In [32]:
support_milk = {}
confidence_milk = {}
lift_milk = {}

In [33]:
for comb1, comb2 in rules:
    support_milk[(comb1, comb2)] = calculate_combination_probability(basket_encoded, comb1 + comb2)
    confidence_milk[(comb1, comb2)] = support_milk[(comb1, comb2)] / calculate_combination_probability(basket_encoded, comb1)
    lift_milk[(comb1, comb2)] = confidence_milk[(comb1, comb2)] / calculate_combination_probability(basket_encoded, comb2)

In [34]:
lift_milk

{(('cooking oil', 'eggs', 'frozen vegetables'), ('milk',)): 3.6002004635720097,
 (('burgers', 'whole wheat rice', 'soup'), ('milk',)): 4.896272630457934,
 (('cooking oil', 'frozen vegetables', 'soup'), ('milk',)): 3.0601703940362084,
 (('chicken', 'frozen vegetables', 'soup'), ('milk',)): 5.246006389776358,
 (('mineral water', 'frozen vegetables', 'soup'),
  ('milk',)): 3.7044167927806737,
 (('tomatoes', 'mineral water', 'frozen vegetables', 'soup'),
  ('milk',)): 5.3552981895633645}

### Class Implementation of Association Rules and Association Trees

In [35]:
from dataclasses import dataclass

In [36]:
@dataclass
class AssociationRule:
    if_: tuple
    then_: tuple
    support: float
    confidence: float
    lift: float

In [37]:
class AssociaitionRulesModel:
    def __init__(self, support_lower: float, confidence_lower: float, lift_lower: float, n_products=3):
        self.support_lower = support_lower
        self.confidence_lower = confidence_lower
        self.lift_lower = lift_lower
        self.n_products = n_products
    
    @staticmethod
    def generate_permutations(columns: list, n: int):
        if n == 1:
            return [(k,) for k in columns]
        permutations = []
        for i, c in enumerate(columns):
            permutations.extend([(c,) + p for p in generate_permutations(columns[i+1:], n-1)])
        return permutations

    @staticmethod
    def generate_all_possible_combinations(columns: list, n: int):
        combinations = []
        for i in range(1, n+1):
            combinations.extend(generate_permutations(columns, i))
        return combinations 
    
    @staticmethod
    def calculate_combination_probability(df: pd.DataFrame, combination: tuple):
        logic = None
        for product in combination:
            if logic is None:
                logic = df[product] == 1
                continue
            logic &= (df[product] == 1)
        return len(df[logic]) / len(df)
    
    def calculate_all_probabilities(self, X: pd.DataFrame):
        self.probabilities_ = {}
        for combination in self.all_possible_combinations_:
            self.probabilities_[frozenset(combination)] = calculate_combination_probability(X, combination)
        return self.probabilities_
    
    def generate_all_rules(self):
        rules = []
        rules_generator = list(filter(lambda x: len(x) > 1, self.all_possible_combinations_))
        for combination in rules_generator:
            rules.extend(self.generate_combination_rules(combination))
        self.all_rules_ = list(set(rules))
        return self.all_rules_
    
    def generate_combination_rules(self, combination: tuple):
        possible_comb = self.generate_all_possible_combinations(combination, self.n_products)
        rules = []
        for if_ in possible_comb:
            for then_ in possible_comb:
                if set(if_).intersection(set(then_)).__len__() != 0:
                    continue
                rules.append((if_, then_))
        return rules
    
    def calculate_rule_metric(self, rule: tuple):
        if_, then_ = rule
        support = self.probabilities_[frozenset(if_ + then_)]
        confidence = 0
        lift = 0
        if self.probabilities_[frozenset(if_)] != 0:
            confidence = self.probabilities_[frozenset(if_ + then_)] / self.probabilities_[frozenset(if_)]
            if self.probabilities_[frozenset(then_)] != 0:
                lift = self.probabilities_[frozenset(if_ + then_)] / (self.probabilities_[frozenset(if_)] * self.probabilities_[frozenset(then_)])
        association_rule = AssociationRule(if_=if_, then_=then_, support=support, confidence=confidence, lift=lift)
        return association_rule
    
    def calculate_metrics(self):
        self.association_metrics_ = []
        for rule in self.all_rules_:
            self.association_metrics_.append(self.calculate_rule_metric(rule))
        return self.association_metrics_
    
    def association_rules(self):
        return list(filter(lambda x: x.support > self.support_lower and x.confidence > self.confidence_lower and x.lift > self.lift_lower, self.association_metrics_))

    def fit(self, X: pd.DataFrame):
        self.feature_names_ = X.columns
        self.all_possible_combinations_ = self.generate_all_possible_combinations(X.columns, self.n_products)
        self.calculate_all_probabilities(X)
        self.generate_all_rules()
        self.calculate_metrics()
        self.association_rules_ = self.association_rules()
    
    def accuracy(self, Xtest: pd.DataFrame):
        test_rules = []
        for rule in self.association_rules_:
            if_ = rule.if_; then_ = rule.then_
            support = self.calculate_combination_probability(Xtest, if_ + then_)
            if_prob = self.calculate_combination_probability(Xtest, if_)
            then_prob = self.calculate_combination_probability(Xtest, then_)
            confidence = support / if_prob if if_prob != 0 else 0
            lift = confidence / then_prob if then_prob != 0 else 0
            test_rule = AssociationRule(if_, then_, support, confidence, lift)
            test_rules.append(test_rule)
        return test_rules
            

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
train, test = train_test_split(basket_encoded)

### Adding Decision Trees to help with Association Rules Selection

In [40]:
class AssociationTree(AssociaitionRulesModel):
    def __init__(self, support_lower: float, confidence_lower: float, lift_lower: float, n_products=3):
        super().__init__(support_lower, confidence_lower, lift_lower, n_products)
        self.probabilities_ = {}
    
    @staticmethod
    def get_association_rules_from_tree(tree: DecisionTreeClassifier, feature_names: list, out: tuple):
        left      = tree.tree_.children_left
        right     = tree.tree_.children_right
        threshold = tree.tree_.threshold
        features  = [feature_names[i] for i in tree.tree_.feature]

        # get ids of child nodes
        idx = np.argwhere(left == -1)[:,0]     

        def recurse(left, right, child, lineage=None):
            if lineage is None:
                buy = np.argmax(tree.tree_.value[child]) 
                lineage = [buy]
            if child in left:
                parent = np.where(left == child)[0].item()
                split = 'l'
            else:
                parent = np.where(right == child)[0].item()
                split = 'r'

            lineage.append((parent, split, threshold[parent], features[parent]))

            if parent == 0:
                lineage.reverse()
                return lineage
            else:
                return recurse(left, right, parent, lineage)
        
        rules = []

        currule = tuple()
        for child in idx:
            for node in recurse(left, right, child):
                if isinstance(node, np.int64):
                    if node == 1:
                        rules.append((currule, out))
                    currule = tuple()
                    continue
                if node[1] == 'r':
                    currule = (node[3],) + currule
        return rules
    
    def generate_feature_name_rules(self, X, feature_name):
        tree = DecisionTreeClassifier(max_depth=self.n_products)
        y = X[feature_name]
        Xtree = X.drop(feature_name, axis=1)
        tree.fit(Xtree, y)
        return self.get_association_rules_from_tree(tree, tree.feature_names_in_, out=(feature_name,))

    def generate_all_rules(self, X: pd.DataFrame):
        self.all_rules_ = []
        for feature_name in self.feature_names_:
            rules = self.generate_feature_name_rules(X, feature_name)
            self.all_rules_.extend(rules)
    
    def calculate_rule_metric(self, rule: tuple):
        if_, then_ = rule
        if frozenset(if_ + then_) not in self.probabilities_.keys():
            p = self.calculate_combination_probability(self._X, if_ + then_)
            self.probabilities_[frozenset(if_ + then_)] = p
        if frozenset(if_) not in self.probabilities_.keys():
            p = self.calculate_combination_probability(self._X, if_)
            self.probabilities_[frozenset(if_)] = p
        if frozenset(then_) not in self.probabilities_.keys():
            p = self.calculate_combination_probability(self._X, then_)
            self.probabilities_[frozenset(then_)] = p
        support = self.probabilities_[frozenset(if_ + then_)]
        confidence = 0
        lift = 0
        if self.probabilities_[frozenset(if_)] != 0:
            confidence = self.probabilities_[frozenset(if_ + then_)] / self.probabilities_[frozenset(if_)]
            if self.probabilities_[frozenset(then_)] != 0:
                lift = self.probabilities_[frozenset(if_ + then_)] / (self.probabilities_[frozenset(if_)] * self.probabilities_[frozenset(then_)])
        association_rule = AssociationRule(if_=if_, then_=then_, support=support, confidence=confidence, lift=lift)
        return association_rule
    
    def fit(self, X: pd.DataFrame):
        self._X = X.copy()
        self.feature_names_ = X.columns
        self.generate_all_rules(X)
        self.calculate_metrics()
        self.association_rules_ = self.association_rules()

In [41]:
basket_encoded.columns.__len__()

25

In [42]:
assotree = AssociationTree(0.001, 0.5, 1, 4)
assotree.fit(train)

In [43]:
asso = AssociaitionRulesModel(0.001, 0.5, 1, 5)
asso.fit(train)

In [44]:
normal_model = asso.accuracy(test)

In [45]:
tree_model = assotree.accuracy(test)

In [46]:
def get_average_parameters(list_rules):
    avg_support = sum([x.support for x in list_rules]) / len(list_rules)
    avg_confidence = sum([x.confidence for x in list_rules]) / len(list_rules)
    avg_lift = sum([x.lift for x in list_rules]) / len(list_rules)
    return {
        'support': avg_support,
        'confidence': avg_confidence,
        'lift': avg_lift
    }

In [47]:
get_average_parameters(normal_model)

{'support': 0.0009856706115035836,
 'confidence': 0.3536577490468407,
 'lift': 1.6300696542934865}

In [48]:
get_average_parameters(tree_model)

{'support': 0.002307438742995275,
 'confidence': 0.3537413244650087,
 'lift': 1.783987236891221}